In [1]:
import numpy as np

In [2]:
class SineCosineOptimization():
    def __init__(self):
        ...
        
    def compile(self,obj,population_size,dimension,lb,ub,max_iter,a=2,optimization="minimize"):
        self.objective_function = obj
        self.dimension = dimension
        self.lb=lb
        self.ub=ub
        self.a=a
        self.max_iter = max_iter
        self.optimization = optimization
        self.population_size=population_size
    
    def __initialize_population(self,population_size,dim,lb,ub):
        return lb+np.random.rand(population_size,dim)*(ub-lb)
    
    def __update_R(self,current_iter,max_iter,a):
        r1 = a-a*(current_iter/max_iter)
        r2 = 2*np.pi*np.random.rand()
        r3 = np.random.rand()
        r4 = np.random.rand()
    
        return r1,r2,r3,r4
    
    def __update_population(self,populations,fittest_candidate,r1,r2,r3,r4,lb,ub):
        if r4 < 0.5:
            populations = populations + r1*np.sin(r2)*np.abs(r3*fittest_candidate-populations)
        else:
            populations = populations + r1*np.cos(r2)*np.abs(r3*fittest_candidate-populations)
        
        return np.clip(populations,lb,ub)
    
    def calculate_optimal_value(self):

        #Initialize the variable for  fittest candidate
        fittest_candidate=None
        candidate_evaluations = np.empty([self.population_size])

        #Initialize the random population
        populations = self.__initialize_population(self.population_size,self.dimension,self.lb,self.ub)

        ## This is the iteration loop.
        for i in range(self.max_iter):

            ## This loop will go through the all population of candidate and process them one by one
            for j in range(populations.shape[0]):
                candidate_evaluations[j]=self.objective_function(populations[j,:])


            ## Determining the fittest candidate
            if self.optimization == "maximize":
                fittest_indice = np.argmax(candidate_evaluations)
            elif self.optimization == "minimize":
                fittest_indice = np.argmin(candidate_evaluations)


            ## Updating the fittest candidate

            if fittest_candidate == None:
                fittest_candidate = (populations[fittest_indice,:],candidate_evaluations[fittest_indice])
            else:
                current_fittest = (populations[fittest_indice,:],candidate_evaluations[fittest_indice])
                if self.optimization == "maximize" and fittest_candidate[1] < current_fittest[1]:
                    fittest_candidate = current_fittest

                if self.optimization == "minimize" and fittest_candidate[1] > current_fittest[1]:
                    fittest_candidate = current_fittest

            ## Updating the value of r1,r2,r3, and r4
            r1,r2,r3,r4 = self.__update_R(i,self.max_iter,self.a)

            ## Update the population
            
            for index,value in enumerate(populations):
                populations[index:] = self.__update_population(value,fittest_candidate[0],r1,r2,r3,r4,self.lb,self.ub)

    

        return fittest_candidate
    
    def one_step_update(self,populations,pop_id,iteration,fittest_candidate=None):
        
        # Initialize candidate evaluation variables
        candidate_evaluations = np.empty([self.population_size])
        
        # Calculate the candidate evaluation using objective function
        for j in range(populations.shape[0]):
            candidate_evaluations[j]=self.objective_function(populations[j,:])
        
         ## Determining the fittest candidate
        if self.optimization == "maximize":
            fittest_indice = np.argmax(candidate_evaluations)
        elif self.optimization == "minimize":
            fittest_indice = np.argmin(candidate_evaluations)
        if fittest_candidate == None:
            fittest_candidate = (populations[fittest_indice,:],candidate_evaluations[fittest_indice])
        
        r1,r2,r3,r4 = self.__update_R(iteration,self.max_iter,self.a)
        
        populations[pop_id,:] = self.__update_population(populations[pop_id,:].reshape(1,-1),fittest_candidate[0],r1,r2,r3,r4,self.lb,self.ub)
        
        return populations
        

### Testing the Algorithm

In [5]:
from benchmarking.benchmark import *
function=[Sphere,Schwefel_2_22,Schwefel_1_2,Rosenbrock,Step,Quartic_with_noise,Ackley,SumSquare]

In [7]:
for i in function:
    func=i()
    values=np.empty(30)
    for l in range(30):
        lb = np.array([func.range[0]]).repeat(func.dim)
        ub = np.array([func.range[1]]).repeat(func.dim)
        sca = SineCosineOptimization()
        sca.compile(func.get_algorithm(),population_size=100,dimension=func.dim,lb=lb,ub=ub,max_iter=100,optimization = "minimize")
        values[l]=sca.calculate_optimal_value()[-1]
    print(f"{func.name} ==> Min = {np.min(values)} Max = {np.max(values)} Mean = {np.mean(values)}")

Sphere ==> Min = 0.0 Max = 1.5673105201673e-13 Mean = 5.224368400558777e-15
Schwefel_2_22 ==> Min = -1.2141690325736208e+28 Max = -3.080013647387586e+20 Mean = -7.226968549379549e+26
Schwefel_1_2 ==> Min = 0.0 Max = 6.52373870690192e-07 Mean = 2.1745795689673065e-08
Rosenbrock ==> Min = 9.04652345598613e-05 Max = 28.901853246259797 Mean = 6.351438905476915
Step ==> Min = 8.770784639940323e-07 Max = 4.038026920094468 Mean = 0.15171552107183753
Quartic_with_noise ==> Min = 8.409971701426509 Max = 10.569960641863439 Mean = 9.60213716487096
Ackley ==> Min = 4.440892098500626e-16 Max = 5.666267455239904e-11 Mean = 1.88989564738525e-12
SumSquare ==> Min = 8.303682220248124e-290 Max = 5.449512270508335e-13 Mean = 3.7432702969413885e-14
